In [1]:
# -*- coding: UTF-8 -*-
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.layers import Layer, LSTM, Input, Embedding, Dense, Activation, Flatten,concatenate
import tensorflow.keras.backend as K

from tensorflow.keras.activations import *

from tensorflow.keras.models import Sequential,Model

from tensorflow.keras import layers

import numpy as np

import matplotlib.pyplot as plt


from tensorflow.data import Dataset

# import keras_tuner as kt

from tensorflow.keras.layers import TextVectorization


In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

## 数据预处理

### 下载并预览数据

In [ ]:
data = keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = data.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print( x_train.shape )
print( x_test.shape )
print( y_train.shape )
print( y_test.shape )

In [ ]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

class_names[y_train[0]]

In [ ]:
plt.imshow(x_train[0])

###  使用 tensorflow 自带的预处理工具 

#### Dataset


In [ ]:
a_tensor = tf.constant([[1] * 4, [2] * 4])
print(a_tensor.get_shape())  # prints ( 2, 4)

In [ ]:
b=[[1, 1, 1, 1, 1],
   [2, 2, 2, 2]
]

b_tensor = tf.constant(b)

print(b_tensor)

In [ ]:
dict_a = {}

k = tf.constant('hello word').numpy()

dict_a[k] = 1

dict_a.keys()

In [ ]:
tf_array=tf.constant([[[1, 3], [2, 3]],
            [[2, 1], [1, 2]],
            [[3, 3], [3, 2]]], shape=(3, 2, 2))


tf_array.numpy()

In [ ]:
# Two tensors can be combined into one Dataset object.
features = tf.constant([[1, 3], 
                        [2, 1], 
                        [3, 3]]) # ==> 3x2 tensor
labels = tf.constant(['A', 'B', 'A']) # ==> 3x1 tensor
dataset = Dataset.from_tensor_slices((features, labels))  # 把tensor 的第一个维度作为样本 id 维度, 按照此维度拼接样本

list(dataset.as_numpy_iterator()) 

In [ ]:
# Both the features and the labels tensors can be converted
# to a Dataset object separately and combined after.
features_dataset = Dataset.from_tensor_slices(features)
labels_dataset = Dataset.from_tensor_slices(labels)
dataset = Dataset.zip((features_dataset, labels_dataset))

list(dataset.as_numpy_iterator())

In [2]:
# A batched feature and label set can be converted to a Dataset
# in similar fashion.
batched_features = tf.constant([[[1, 3], 
                                 [2, 3]],
                                [[2, 1], 
                                 [1, 2]],
                                [[3, 3], 
                                 [3, 2]]], shape=(3, 2, 2))
batched_labels = tf.constant([['A', 'A'],
                              ['B', 'B'],
                              ['A', 'B']], shape=(3, 2))
dataset = Dataset.from_tensor_slices((batched_features, batched_labels))

dataset_batch = dataset.batch(2)

for i,element in enumerate(dataset):
    
    print(i)
    
    print(element[0])
    
#     print(element[1].shape)
    
    print(element[1].numpy())

0
tf.Tensor(
[[1 3]
 [2 3]], shape=(2, 2), dtype=int32)
(2,)
[b'A' b'A']
1
tf.Tensor(
[[2 1]
 [1 2]], shape=(2, 2), dtype=int32)
(2,)
[b'B' b'B']
2
tf.Tensor(
[[3 3]
 [3 2]], shape=(2, 2), dtype=int32)
(2,)
[b'A' b'B']


In [3]:
for i,element in enumerate(dataset_batch):
    
    print(i)
    
    print(element)
    


0
(<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[1, 3],
        [2, 3]],

       [[2, 1],
        [1, 2]]])>, <tf.Tensor: shape=(2, 2), dtype=string, numpy=
array([[b'A', b'A'],
       [b'B', b'B']], dtype=object)>)
1
(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[3, 3],
        [3, 2]]])>, <tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'A', b'B']], dtype=object)>)


In [ ]:
dataset_batch.numpy()

In [5]:

dataset_batch = dataset.batch(1)

for i,element in enumerate(dataset_batch):
    
    print(i)
    
    print(element)
    


0
(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[1, 3],
        [2, 3]]])>, <tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'A', b'A']], dtype=object)>)
1
(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[2, 1],
        [1, 2]]])>, <tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'B', b'B']], dtype=object)>)
2
(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[3, 3],
        [3, 2]]])>, <tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'A', b'B']], dtype=object)>)


In [6]:
dataset_batch = dataset.batch(2)

for i,element in enumerate(dataset_batch):
    
    print(i)
    
    print(element)

0
(<tf.Tensor: shape=(2, 2, 2), dtype=int32, numpy=
array([[[1, 3],
        [2, 3]],

       [[2, 1],
        [1, 2]]])>, <tf.Tensor: shape=(2, 2), dtype=string, numpy=
array([[b'A', b'A'],
       [b'B', b'B']], dtype=object)>)
1
(<tf.Tensor: shape=(1, 2, 2), dtype=int32, numpy=
array([[[3, 3],
        [3, 2]]])>, <tf.Tensor: shape=(1, 2), dtype=string, numpy=array([[b'A', b'B']], dtype=object)>)


In [ ]:
label2 = tf.constant([[1],
                      [2],
                      [3]])


for feature,label,label2 in zip(batched_features, batched_labels, label2):
    
    print(feature)
    
    print(label)
    
    print(label2)

In [6]:
features = tf.constant( [
                        [1, 3, 2, 3],
                        [2, 1, 1, 2],
                        [3, 3, 3, 2],
                        [0, 0, 0, 2]
                        ], shape=(4, 4))

dataset = Dataset.from_tensor_slices(features).batch(2)

source_in = dataset.map(lambda batch_text: batch_text[:, ::-1], num_parallel_calls=tf.data.AUTOTUNE)

target_out = dataset.map(lambda batch_text: batch_text[:, 1:], num_parallel_calls=tf.data.AUTOTUNE)

target_in = dataset.map(lambda batch_text: batch_text[:, :-1], num_parallel_calls=tf.data.AUTOTUNE)


print('source_in')

for ele in source_in:
    
    print(ele)


print('target_out')

for ele in target_out:
    
    print(ele)

print('target_in')
    
for ele in target_in:
    
    print(ele)
    



source_in
tf.Tensor(
[[3 2 3 1]
 [2 1 1 2]], shape=(2, 4), dtype=int32)
tf.Tensor(
[[2 3 3 3]
 [2 0 0 0]], shape=(2, 4), dtype=int32)
target_out
tf.Tensor(
[[3 2 3]
 [1 1 2]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[3 3 2]
 [0 0 2]], shape=(2, 3), dtype=int32)
target_in
tf.Tensor(
[[1 3 2]
 [2 1 1]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[3 3 3]
 [0 0 0]], shape=(2, 3), dtype=int32)


#### Strings


In [ ]:
import re 

text = '&apos The English ##AT##-##AT## abc10 56 71 German 4x4 abc-10 Pro Dictionary contains over 50,813 words and 23,343 articles presented in rich ##AT##-##AT## text format articles .'

# 删除单词的规则
remove_words = r'(##AT##-##AT##|&apos|&quot)'


# 需要删除的标点符号, 包含中文符号
punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~“”？，！【】（）、。：；’‘……￥·"""
punctuation = punctuation.replace("'", "")  # 英语中有', 不删除 '
punctuation = punctuation.replace("-", "")  #
remove_punc = r'[%s]' % re.escape(punctuation)

# 删除单独的数字字符
# eg. 'avc 10 abc-10 abc10' -> 'avc  abc-10 abc10'

remove_digits = r'^(\d+ )+|( \d+)+ |(\d+)$'

text = tf.strings.regex_replace(text, remove_words, '')

text = tf.strings.regex_replace(text, remove_punc, ' ') 

text = tf.strings.regex_replace(text, remove_digits, ' ')

print(text)

In [3]:
text = '11 999 avc 10 abc-10 10-abc abc10 50 813 aa 4x4 20'


#  A=( \d+) 空格+至少1个数字 为匹配字符A ,
#  B='A+ ' 至少1个A + 空格 为匹配字符B
# remove_digits = r'( \d+)+ '

# 考虑数字可能出现在句首
# remove_digits = r'^(\d+ )+|( \d+)+ '

# 考虑数字可能出现在句首和句末 
remove_digits = r'^(\d+ )+|( \d+)+ |(\d+)$'

text = tf.strings.regex_replace(text, remove_digits, ' ')

print(text)


tf.Tensor(b' avc abc-10 10-abc abc10 aa 4x4  ', shape=(), dtype=string)


#### Tokenizer

## 序列（Sequential）建模

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

In [ ]:
model.summary()

In [ ]:
model = Sequential( [Flatten( input_shape=[28, 28] ),
                     Dense( 100, activation='relu' ),
                     Dense( 10, activation='softmax' )
                    ] )

In [ ]:
model.summary()

In [ ]:
model.layers[1].name

In [ ]:
weights, biases = model.layers[1].get_weights()

In [ ]:
weights

In [ ]:
biases

In [ ]:
print( weights.shape )
print( biases.shape )

## 函数（Functional）建模

### 例子1 

In [ ]:
inputs = Input( shape=[28,28] )
x = Flatten()( inputs )
x = Dense(100, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

basic_model = Model( inputs=[inputs], outputs=[output] )

In [ ]:
basic_model.summary()


### 例子2

In [ ]:

def base_model(m):

    # 建立模型第一层的 keras Tensor，只能是Input 或者从 别的地方传来的 tensor from keras.layer
    X = Input(shape=(32,), name='X') # 此时 shape(None,32) ；真正训练时为 shape(m,32)

    init_state = Input(shape=(32,), name='init_state')

    #1.
    # b= K.zeros((100,32))
    # b = Input( name='input_b',tensor=K.zeros((100,32)))  #<tf.Variable 'Variable:0' shape=(100, 32) dtype=float32>
    # 喂入模型的一个 batch的样本的数目 m 此时是未知的，因此为None ，
    # 所以模型的第一层中，所有和 m 相关的tensor 都要使用 Input，相当于tensorflow的一个占位符，其他方法不work:
    #
    # 反例1:
    # b=K.zeros((init_state.shape[0],32)) # init_state.shape[0]=None，K.zeros 的shape参数 中不能出现None
    #
    # 反例2:
    # b= K.zeros((m,32)) # m 由参数传入，m=100
    # b = Input( name='input_b',tensor=K.zeros((m,32))) # <tf.Variable 'Variable:0' shape=(100, 32) dtype=float32>
    # 虽然成功建立的 tensor 但是和 来源于 keras.layer 的a (shape: (None,32)) 无法进行 axis=1的拼接
    # 即报错： ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 32), (100, 32)]

    #2.
    # 连接的所有tensor 必须来源于 keras.layer（血统纯正），否则无法生成计算图
    # 下面两种方法都报 AttributeError: 'NoneType' object has no attribute '_inbound_nodes'
    #M1
    # init_state_2 = K.constant(np.zeros((1,64)), dtype='float32') 
    #M2
    # init_state_2=K.ones((1,64))
    
    #M3 使用 keras.layer.Reshape 转换，但是维度不对,会自动的在全面加上一维
    

    concat = Concatenate(axis=1)([X, init_state])

#     concat2= Concatenate(axis=1)([concat, b])
#     concat2 = Concatenate(axis=0)([concat, init_state_2])

    dense = Dense(1, name='dense_1')(concat)

    print(dense)
    
    model= Model(inputs=[ X , init_state ], outputs=dense)

    return model


M=100
model = base_model(M)



In [ ]:
X=np.random.randint(0,10,size=[M,32])
init_state=np.zeros((M,32))
# b_train=K.zeros((M,32)) #ValueError: If your data is in the form of symbolic tensors, you should specify the `steps_per_epoch` argument (instead of the `batch_size` argument, because symbolic tensors are expected to produce batches of input data).

x_train = [X, init_state]
y_train=np.random.randint(low=0,high=2,size=[M,1])


model.compile(loss='binary_crossentropy', optimizer='sgd')

batch_size=10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5
          )

## 子类化（Subclassing）建模

官方教程

https://keras.io/guides/making_new_layers_and_models_via_subclassing/


可训练的权重变量(有状态)必须作为类的变量并且被 add_weight(), 如果嫌麻烦也可以直接在 init 中写 Keras layer

### 例子1

In [ ]:
w = tf.Variable([[1.], [2.]])
x = tf.constant([[3., 4.]])

tf.matmul(w, x)  # 矩阵乘法 相当于 np.dot()

tf.sigmoid(w + x)


In [ ]:
class Linear(Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        ) # 可训练的权重变量(有状态) 作为类的变量并且被 add_weight()
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

N = 2
    
x = tf.ones((N, 2))

linear_layer = Linear(4)

y = linear_layer(x)

print(y)

### 例子2

把 Keras Layer 当做类变量, 与函数建模很类似，好处是： 不用写 输入层(Input) 和 在最后定义 model 指明输入输出层


In [ ]:
class Linear(Layer):  
  
    def __init__( self, units=32, **kwargs ):

        super().__init__(**kwargs)
        self.dense_layer = Dense(units) 

    def call(self, inputs):

        x = Flatten()(inputs)
        x = self.dense_layer(x)
        return x

In [ ]:
model = Linear(4)

model

# model.layers

In [ ]:
x = tf.ones((2, 2))
linear_layer = Linear(4)
y = linear_layer(x)
print(y)

**Layer 和 Model 的区别**


通常，您会使用 Layer 类来定义内部计算块，并使用 Model 类来定义外部模型，即您将训练的对象。

例如，在 ResNet50 模型中，您会有几个子类化 Layer 的 ResNet 块，以及一个包含整个 ResNet50 网络的 Model。

Model 类具有与 Layer 相同的 API，但有如下区别：

* 它会公开内置训练、评估和预测循环（model.fit()、model.evaluate()、model.predict()）。
* 它会通过 model.layers 属性公开其内部层的列表。
* 它会公开保存和序列化 API（save()、save_weights()…）

实际上，Layer 类对应于我们在文献中所称的“层”（如“卷积层”或“循环层”）或“块”（如“ResNet 块”或“Inception 块”）。

同时，Model 类对应于文献中所称的“模型”（如“深度学习模型”）或“网络”（如“深度神经网络”）。

因此，如果您想知道“我应该用 Layer 类还是 Model 类？”，请问自己：我是否需要在它上面调用 fit()？我是否需要在它上面调用 save()？如果是，则使用 Model。如果不是（要么因为您的类只是更大系统中的一个块，要么因为您正在自己编写训练和保存代码），则使用 Layer。

### 例子3


In [ ]:
#   By XRH in 2019.12.28
# 使用 keras.layer 搭建神经网络 不太灵活：
# （1）keras.layer 提供的结构太少
# （2）要使用 后端提供的丰富的函数 还需要使用 Lambda层进行包装，较繁琐
# 因此，我们 使用 后端的函数来 定义属于自己的层


class ConcatDense(Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(ConcatDense, self).__init__(**kwargs)

    def build(self, input_shape):
        assert isinstance(input_shape, list)
        # 为该层创建一个可训练的权重
        self.w = self.add_weight(name='w',
                                      shape=(input_shape[0][1]+input_shape[1][1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        
        self.b = self.add_weight( name='b',
                                    shape=(self.output_dim,),
                                        initializer='zeros',
                                        trainable=False)
        
        self.input_shapes=input_shape
        
        super(ConcatDense, self).build(input_shape)  # 一定要在最后调用它

    def call(self, inputs):
        assert isinstance(inputs, list)
        
        print('input_shapes:', self.input_shapes)
        
        X, init_state = inputs # X shape: (None,32) , init_state shape: (None,32)
        
        print(X)
        print(init_state)
        
        concat=K.concatenate([X,init_state],axis=1) # shape(None, 64)
        print(concat) 
        
        z = K.dot(concat, self.w) + self.b
        # concat shape (None, 64), self.w shape(64, 10) -> shape (None, 10)
        print(z)
        
        a = relu(z) # 激活函数
        print(a)
        
        return a

    def compute_output_shape(self, input_shape):
        assert isinstance(input_shape, list)
        shape_X, shape_init_state = input_shape
        return (shape_X[0], self.output_dim)


    
def test_model():

    # 建立模型第一层的 keras Tensor，只能是Input 或者从 别的地方传来的 tensor from keras.layer
    X = Input(shape=(32,), name='X') # 此时 shape(None,32) ；真正训练时为 shape(m,32)

    init_state = Input(shape=(32,), name='init_state')

    layer1 = ConcatDense(10, name='layer1')([ X , init_state] )
    
    layer2 = Dense(1, name='layer2')(layer1)
    
    model= Model(inputs=[ X , init_state ], outputs=layer2)

    return model


model = test_model()



In [ ]:

N=10 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]
y_train=np.random.randint(low=0,high=2,size=[N,1]) # 取值范围 [0, 2)

# y_train

model.compile(loss='binary_crossentropy', optimizer='sgd')

batch_size=10
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=5
          )

## 模型调试

### tensor 的直接计算

#### tensoflow 1.x 

采用静态图的方法，需要 使用会话来 run 计算图，其才会执行

In [ ]:
import tensorflow.compat.v1 as tf1
import numpy as np 

tf1.disable_v2_behavior()
 
 
a=np.array(
[[1,2,3,4,5],
[1,2,3,4,5],
[1,2,3,4,5]]    
)
# a=np.array([[2.29982214e-10,1.05035841e-03,1.04089566e-04,9.98845458e-01,
#   5.10124494e-08,5.03688757e-10,2.52189380e-10,1.18713073e-09,
#   2.30988277e-08,2.21948682e-08,1.48340121e-07]])



input = tf1.constant(a)
k = 3
output = tf1.nn.top_k(input, k).indices


# one_hot=one_hot_tensor(output,11)
# one_hot=one_hot[0]
# # one_hot[0].shape
# one_hot=K.reshape(one_hot,(1,one_hot.shape[0],one_hot.shape[1]))
# # one_hot.shape
# one_hot_permute=K.permute_dimensions(one_hot,(1,0,2))
# one_hot_permute.shape

with tf1.Session() as sess:
    
    print(sess.run(input))
    print(sess.run(output))
#     print(sess.run(one_hot))
#     print(sess.run(one_hot_permute))


In [ ]:
output

#### tensorflow 2.X

TensorFlow的Eager模式，也可以看做是动态图模型。该模型下不需要先构造图，然后再使用Session.run()，而是可以得到即时的反馈。这样在研究和开发时会更加符合直觉。

https://zhuanlan.zhihu.com/p/47201474

In [ ]:
import tensorflow as tf

print(tf.__version__)

#检查是否开启 eager 模式, 在使用了tensorflow.compat.v1 后就会将 eager 模式退出
tf.executing_eagerly() 

tf.config.experimental_run_functions_eagerly(True) # 没有起到作用



In [ ]:

rank_2_tensor = tf.constant([[1, 2],
                             [3, 4],
                             [5, 6]], dtype=tf.float16)
print(rank_2_tensor)


In [ ]:

a = tf.constant([[10,10],
                 [11.,1.]])

x = tf.constant([[1.,0.],
                 [0.,1.]])

b = tf.Variable(12.)

c = np.ones((2))

y = tf.matmul(a, x) + b 

print(y)

y2 = y+c

print(y2)

##### Keras

In [ ]:
def top_k_tensor(input, k):
    """
    返回张量 input, 第0个维度的 topk 个元素的标号

    :param input:
    :param k:
    :return:
    """

    return tf.nn.top_k(input,k).indices

a = np.array(
    [[1, 2, 3, 4, 5],
     [1, 2, 2, 2, 2],
     [1, 3, 3, 3, 6]]
)
k = 3

# 静态图下要使用 eval 才能触发计算
print('top_k_tensor: \n', K.eval(top_k_tensor(a,k)))

# 动态图下 直接打印即可
print(top_k_tensor(a,k))

In [ ]:
a= np.array([[1,2,3,4]])
layer1 = Dense(1)

out = layer1(a)
out 
print(out) # 直接执行

###  查看所有层(layer)

In [ ]:
model.summary()

In [ ]:
# 查看所有的 layer 

model.layers



In [ ]:
from tensorflow.keras.utils import plot_model 

plot_model(model)

### 输出指定层的参数

In [ ]:
model.layers[2]

model.layers[2].name

model.get_layer('layer1')

W , b = model.layers[2].get_weights()

np.shape(W)
np.shape(b)


In [ ]:
# 输出该层的所有参数
for weight in model.get_layer('layer1').weights:
    print(weight.name, weight.shape)


In [ ]:
# 输出所有层的所有参数

for i in range(len(model.layers)):
    
    for weight in model.layers[i].weights:
        
        print(weight.name, weight.shape)


In [ ]:
# 查看所有层 和 所有层的输出

for index in range(len(model.layers)):
    print(model.get_layer(index=index).name, model.get_layer(index=index).output_shape)


### 模型(计算图)中间层的计算结果

In [ ]:
model.layers[2].name

model.layers[2].input

model.layers[2].output

#### 使用 K.function()

##### 例子1

In [ ]:

mid_func = K.function( inputs=[ model.layers[0].input, model.layers[1].input ] ,outputs=[model.layers[2].output] )

N=5 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]

res = mid_func([X, init_state])

np.shape(res)


**使用K.function()可以快速检查层的正确性**

##### 例子2 LSTM

In [ ]:


N = 2
max_length = 4
n_embedding=20
n_vocab=100
n_a = 10


word_embedding_layer = Embedding(n_vocab, n_embedding, input_length=max_length  , name='word_embedding')
lstm_layer = LSTM(units=n_a, return_sequences=True, return_state=True)


word_list = Input(shape=(max_length))  

mask = (word_list != 0) # shape(N,max_length)

word_embedding = word_embedding_layer(inputs=word_list)

out, state_h, state_c = lstm_layer(inputs=word_embedding, mask=mask)
    

f_out = K.function(inputs=[word_list], outputs=out)

f_mask = K.function(inputs=[word_list], outputs=mask)

f_state_h = K.function(inputs=[word_list], outputs=state_h)

f_state_c = K.function(inputs=[word_list], outputs=state_c)


seq_data = np.ones((N, max_length))


# seq_data =np.random.randint(n_vocab,size=(N, max_length))  

res_out = f_out([seq_data])
res_state_h = f_state_h([seq_data])
res_state_c = f_state_c([seq_data])


res_mask = f_mask([seq_data])


In [ ]:
np.shape(res_out)
res_out

In [ ]:
seq_data

np.shape(res_out)
res_mask

In [ ]:
# 把 LSTM 的时间步拆出来, 一步一步执行

# 1.执行第1个时间步

word_list_t = Input(shape=(max_length))  
mask = (word_list_t != 0) 
word_embedding_t = word_embedding_layer(inputs=word_list_t)
out_t, state_h_t, state_c_t = lstm_layer(inputs=word_embedding_t, mask=mask)

f_out_t = K.function(inputs=[word_list_t], outputs=out_t)

# 测试数据
seq_data_0 = np.expand_dims(seq_data[:,0], axis=1)
# np.shape(seq_data_0)

# 推理
res_out_0 = f_out_t([seq_data_0])
res_out_0

res_out[:,0,:]

In [ ]:
# 2.执行第1个时间步，给 LSTM 一个初始隐藏层状态 h0

word_list_t = Input(shape=(max_length))  
h0 = Input(shape=(n_a))
c0 = Input(shape=(n_a))

mask = (word_list_t != 0) 
word_embedding_t = word_embedding_layer(inputs=word_list_t)
h=h0
c=c0
out_t, h_t, c_t = lstm_layer(inputs=word_embedding_t, mask=mask, initial_state=[h, c])

f_out_t = K.function(inputs=[word_list_t, h0, c0], outputs=out_t)
f_h_t = K.function(inputs=[word_list_t, h0, c0], outputs=h_t)
f_c_t = K.function(inputs=[word_list_t, h0, c0], outputs=c_t)

# 测试数据
seq_data_0 = np.expand_dims(seq_data[:,0], axis=1)
# np.shape(seq_data_0)

data_h = np.ones((N, n_a))
# data_h = np.zeros((N, n_a))
data_c = np.zeros((N, n_a))

# 推理
res_out_0 = f_out_t([seq_data_0, data_h, data_c])
res_out_0

# res_out[:,0,:]


res_h_t = f_h_t([seq_data_0, data_h, data_c])
res_h_t


res_c_t = f_c_t([seq_data_0, data_h, data_c])
res_c_t

#### 建立新的 model

使用原来的 layer 建立新的计算图

In [ ]:

mid_model = Model( inputs=[ model.layers[0].input, model.layers[1].input ] ,outputs=[model.layers[2].output] )

N=5 # 10 个样本

X=np.random.randint(0,10,size=[N,32])
init_state=np.zeros((N,32))

x_train = [X, init_state]

res = mid_model.predict([X, init_state])

np.shape(res)

# res


## 编译和拟合模型

### 使用内置的方法

In [ ]:
basic_model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

In [ ]:
basic_model.fit( x_train, y_train, epochs=10, validation_split=0.2 )

In [ ]:
class myCallback( tf.keras.callbacks.Callback ):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.9):
      print("\nReached 90% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

model.fit( x_train, y_train, epochs=20, callbacks=[callbacks] )

### 从头编写训练循环

In [ ]:


# Prepare the training dataset.
batch_size = 64
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = np.reshape(x_train, (-1, 784))
x_test = np.reshape(x_test, (-1, 784))

# Reserve 10,000 samples for validation.
x_val = x_train[-10000:]
y_val = y_train[-10000:]
x_train = x_train[:-10000]
y_train = y_train[:-10000]

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Prepare the validation dataset.
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)



In [ ]:
# Get model
inputs = keras.Input(shape=(784,), name="digits")
x = layers.Dense(64, activation="relu", name="dense_1")(inputs)
x = layers.Dense(64, activation="relu", name="dense_2")(x)
outputs = layers.Dense(10, name="predictions")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
# Instantiate an optimizer to train the model.
optimizer = keras.optimizers.SGD(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Prepare the metrics.
train_acc_metric = keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = keras.metrics.SparseCategoricalAccuracy()

In [ ]:
import time

epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))
    start_time = time.time()

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch_train, training=True)
            loss_value = loss_fn(y_batch_train, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update training metric.
        train_acc_metric.update_state(y_batch_train, logits)

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %d samples" % ((step + 1) * batch_size))

    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))

    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()

    # Run a validation loop at the end of each epoch.
    for x_batch_val, y_batch_val in val_dataset:
        val_logits = model(x_batch_val, training=False)
        # Update val metrics
        val_acc_metric.update_state(y_batch_val, val_logits)
    val_acc = val_acc_metric.result()
    val_acc_metric.reset_states()
    print("Validation acc: %.4f" % (float(val_acc),))
    print("Time taken: %.2fs" % (time.time() - start_time))



## 预测模型

In [ ]:
basic_model.evaluate( x_test, y_test )

In [ ]:
prob = model.predict( x_test[0:1] )
prob

In [ ]:
import numpy as np

np.argmax(prob)

In [ ]:
print( y_test[0] )

In [ ]:
plt.imshow(x_test[0])

In [ ]:
# 训练精度 90% 但是测试精度 87.7%，有过拟合的征兆

### 引进验证集

In [ ]:
data = keras.datasets.fashion_mnist

(x_train_full, y_train_full),(x_test, y_test) = data.load_data()

x_valid, x_train = x_train_full[:5000] / 255.0, x_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [ ]:
class myCallback( tf.keras.callbacks.Callback ):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_acc')>0.9):
      print("\nReached 90% validation accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [ ]:
model = Sequential()

model.add( Flatten(input_shape=[28,28]) )
model.add( Dense(100, activation='relu') )
model.add( Dense(10, activation='softmax') )

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid), 
                     callbacks=[callbacks] )

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D

data = keras.datasets.fashion_mnist

(x_train_full, y_train_full),(x_test, y_test) = data.load_data()

x_train_full = x_train_full.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

x_valid, x_train = x_train_full[:5000] / 255.0, x_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


model = Sequential([Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                    MaxPooling2D(2,2),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Flatten(),
                    Dense(128, activation='relu'),
                    Dense(10, activation='softmax') ])

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid) )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure( figsize=(8,4), dpi=100 )

plt.subplot(1,2,1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from tensorflow.keras.layers import Dropout

model = Sequential([Conv2D(64, (3,3), activation='relu', input_shape=(28,28,1)),
                    MaxPooling2D(2,2),
                    Conv2D(64, (3,3), activation='relu'),
                    MaxPooling2D(2,2),
                    Flatten(),
                    Dropout(0.5),
                    Dense(128, activation='relu'),
                    Dense(10, activation='softmax') ])

model.compile( loss='sparse_categorical_crossentropy',
               optimizer='adam',
               metrics=['accuracy'] )

history = model.fit( x_train,
                     y_train,
                     epochs=20, 
                     validation_data=(x_valid, y_valid) )

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.figure( figsize=(8,4), dpi=100 )

plt.subplot(1,2,1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## 超参数调优 Keras Tuner

**Ref**

[1] https://zhuanlan.zhihu.com/p/156139224

[2] https://www.tensorflow.org/tutorials/keras/keras_tuner

### 例子1 使用序列建模

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
# from kerastuner.tuners import RandomSearch

import keras_tuner as kt

In [ ]:
print( x_train.shape )
print( x_test.shape )
print( y_train.shape )
print( y_test.shape )

In [ ]:
def build_model(hp):
    model = keras.Sequential()
    model.add( Flatten(input_shape=[28,28]) )
    model.add(layers.Dense(units=hp.Int('units',
                                        min_value=32,
                                        max_value=512,
                                        step=32),
                           activation='relu')) 
    model.add(layers.Dense(10, activation='softmax')) 
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    return model

In [ ]:
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy', #优化目标
    max_epochs=20, 
    directory='E:\python package\python-project\DeepLearningApp\LearnKeras\logs',
    project_name='image_classify')

# directory 要使用绝对路径, 否则报 UnicodeDecodeError 真TM坑

In [ ]:
tuner.results_summary()

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train,
             y_train, 
             epochs=5, 
             validation_split=0.2, callbacks=[stop_early]) 

### 例子2 使用函数建模

In [ ]:

def build_model2(hp):

    inputs = Input(shape=[28,28])
    x = Flatten(input_shape=[28,28])(inputs)
    x = Dense(units=hp.Int('units',
                            min_value=50,
                            max_value=512,
                            step=50), activation='relu')(x)
    output = Dense(10, activation='softmax')(x)

    model = Model( inputs=[inputs], outputs=[output] )
    
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate',
                      values=[1e-2, 1e-3, 1e-4])),
        
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy'])
    
    return model


tuner = kt.Hyperband(
    build_model2,
    objective='val_accuracy', #优化目标
    max_epochs=10, 
    directory='E:\python package\python-project\DeepLearningApp\LearnKeras\logs', # directory 要使用绝对路径, 否则报 UnicodeDecodeError 
    project_name='imageclassify') # 每次运行之前, 将 imageclassify 目录删除

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(x_train,
             y_train, 
             epochs=10, 
             validation_split=0.2, callbacks=[stop_early]) 

In [ ]:
tuner.results_summary(3)

In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]

In [ ]:
tuner.get_best_hyperparameters()[0]

In [ ]:
best_model.evaluate( x_test, y_test )

In [ ]:
best_model.summary()

In [ ]:
basic_model.evaluate( x_test, y_test )

In [ ]:
basic_model.summary()

### 例子3